# Importamos librerías y conectamos al drive

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tokenize import word_tokenize 
from   nltk.tokenize import TreebankWordTokenizer 
from   nltk.stem     import SnowballStemmer, WordNetLemmatizer #PorterStemmer
from   nltk.corpus   import stopwords #lista de stop words

nltk.download('wordnet') #me bajo los datos para poder usarlos
nltk.download('punkt')
nltk.download('stopwords')

tokenizer  = TreebankWordTokenizer()
stemmer    = SnowballStemmer("english") #PorterStemmer
lemmatizer = WordNetLemmatizer()

 #Importo los vectorizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import pickle #para guardado de datos
from sklearn.naive_bayes             import MultinomialNB
import os
from sklearn.model_selection import cross_validate
#!pip install varname
#from varname import nameof #para obtener el nombre de variables

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd "/content/drive/Shareddrives/Redes neuronales/TP1"

/content/drive/Shareddrives/Redes neuronales/TP1


In [4]:
!pip install import-ipynb

In [5]:
import import_ipynb
from Helper import *

importing Jupyter notebook from Helper.ipynb
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Cargo los datos

In [6]:
df_train = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/train_data.hdf5")
df_valid = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/valid_data.hdf5")
df_test = pd.read_hdf("/content/drive/Shareddrives/Redes neuronales/TP1/data/test_data.hdf5")

In [7]:
#Cantidad de documentos en train
len(df_train)

549367

In [8]:
df_train.head()

,gold_label,text
pairID,,
1000092795.jpg#0r1c,contradiction,they are inside of a house
1000092795.jpg#0r1e,entailment,two guys are in a yard
1000092795.jpg#0r1n,neutral,They are doing yardwork
1000092795.jpg#1r1c,contradiction,A man is swimming.
1000092795.jpg#1r1e,entailment,Two young white men are near some bushes.


In [9]:
df_train['text'][0]

'they are inside of a house'

In [10]:
df_train['gold_label'].values

array(['contradiction', 'entailment', 'neutral', ..., 'contradiction',
       'entailment', 'neutral'], dtype=object)

In [11]:
df_valid.head()

,gold_label,text
pairID,,
100197432.jpg#4r1c,contradiction,The women are seated in a bistro eating lunch.
100197432.jpg#4r1e,entailment,There are muliple women outdoors.
100197432.jpg#4r1n,neutral,The women are friends.
1026792563.jpg#1r1c,contradiction,A group of women are fishing
1026792563.jpg#1r1e,entailment,A group of women are selling their wares


In [12]:
df_test.head()

,text
pairID,
0,The church has cracks in the ceiling.
1,The church is filled with song.
2,A choir singing at a baseball game.
3,The woman is young.
4,The woman is very happy.


In [13]:
 #df_submission = pd.read_csv("/kaggle/input/sesgos-en-el-dataset-de-snli/submission_sample.csv", index_col="pairID")


In [14]:
#df_submission

In [15]:
text_train = df_train["text"].tolist() #nos quedamos con los textos
labels_train = df_train["gold_label"].tolist() #nos quedamos con la clasificación
text_val = df_valid["text"].tolist()
labels_val = df_valid["gold_label"].tolist()
text_test = df_test["text"].tolist()

In [16]:
#Veamos el balance de clases
Counter(labels_train)
#esta bastante parejo, dataset equilibrado

Counter({'contradiction': 183187, 'entailment': 183416, 'neutral': 182764})

In [17]:
len(labels_val)

9842

In [18]:
len(text_val)

9842

##Mezclo los datos de train y valid para hacer luego cross validation

In [19]:
df_train_valid = pd.concat([df_train,df_valid])
len(df_train_valid)

559209

In [20]:
len(df_train) + len(df_valid)

559209

In [21]:
text_train_valid = df_train_valid["text"].tolist() #nos quedamos con los textos
labels_train_valid = df_train_valid["gold_label"].tolist() #nos quedamos con la clasificación



# Preprocesamiento de texto

A continuación, vamos a ir probando con distintos parámetros de preprocesamiento:

1. is_lc: pone a todo el texto en mínusculas o lower case (lc). 
2. is_lem: lematiza el texto. Lemmatize se reduce la palabra a su raíz pero se asegura de que no pierda su significado. La lemmatización tiene un diccionario predefinido que almacena el contexto de las palabras y verifica que la palabra esté en el diccionario mientras la reduce.
3. is_stop: saca las palabras o stop words de las hipótesis:

In [26]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

4. is_stem: stemming se llama estandarización de texto donde las palabras se reducen a su forma base/raíz. Por ejemplo, palabras como "programador", "programación," programa "se derivarán de" programa ".
5. is_alpha': elimina lo que no son palabras, ej:comas,puntos.

En principio, se pensaría que debido a que se quiere conservar el "contexto" de la hipótesis, no serviría eliminar las stop words ni eliminar todo lo que no son palabras que le dan sentido a la oración. Por otro lado, low case podría serviría para que una palabra escrita con mayúscula sea tenida en cuenta igual a una palbra escrita con minúsculas. En principio, se pensaría que si se eliminan, se podría perder el sentido de la oración ya que una palabra que empieza con mayúscula indica que es el inicio de la oración. Sin embargo, si no se eliminan lo que no son palabras, en principio no habría problema de perder el sentido de la oración ya que seguirían estando los puntos y las comas. 

En cuanto al stemming y lemming. Como con lemmatize se reduce la palabra a su raíz pero se asegura de que no pierda su significado, mientras que con stemming, se reduce a las palabras de tal manera que su forma raíz pierde el significado, a una forma que deja de ser una palabras en inglés adecuada. Ejemplo: crazy -> crazi, early -> earli, available -> avail. En principio, se pensaría que probablemente convenga hacer lemming y no stemming.

##Vemos diferencias en los resultados variando los parámetros del preprocesamiento con los siguientes hiperparámetros:
- alpha: 1
- min_df = 0.000001
- max_df = 0.8
- tf_idf

###Creación de combinaciones de parámetros de preprocesamiento

In [ ]:
#definición de parámetros de preprocesamiento
parameters_preprocessing_specs = {  
    'is_lc': [True, False],
    'is_lem': [True, False],
    'is_stop': [True, False],
    'is_stem': [True, False],
    'is_alpha': [True, False]
}

In [ ]:
#hacemos las combinaciones
def get_parameters_preprocessing():
      for is_lc in parameters_preprocessing_specs['is_lc']:
          for is_lem in parameters_preprocessing_specs['is_lem']:
              for is_stop in parameters_preprocessing_specs['is_stop']:
                  for is_stem in parameters_preprocessing_specs['is_stem']:
                    for is_alpha in parameters_preprocessing_specs['is_alpha']:
                      parameters_preprocessing = {
                          'is_lc': is_lc,
                          'is_lem': is_lem,
                          'is_stop': is_stop,
                          'is_stem': is_stem,
                          'is_alpha': is_alpha
                      }
                      
                      yield parameters_preprocessing #gives a generator function, para evitar muchos llamados

In [ ]:
parameters_preprocessing = pd.DataFrame()

for p in get_parameters_preprocessing():
    df = pd.DataFrame(p, index=[0])
    parameters_preprocessing = parameters_preprocessing.append(df,ignore_index=True) #ponemos combinaciones

parameters_preprocessing 

,is_lc,is_lem,is_stop,is_stem,is_alpha
0,True,True,True,True,True
1,True,True,True,True,False
2,True,True,True,False,True
3,True,True,True,False,False
4,True,True,False,True,True
5,True,True,False,True,False
6,True,True,False,False,True
7,True,True,False,False,False
8,True,False,True,True,True
9,True,False,True,True,False


### Vemos los resultados variando los parámetros entrenando con 300.000 hipotesis del conjunto de train

Probamos con 300.000 hipótesis porque sino era muy costoso computacionalmente entrenar el modelo con todas las hipóteiss variando todos las posibles combinaciones de los parámetros de preprocesamiento del texto.

In [ ]:
len(text_train_valid)

559209

Chequeamos que las primeras 300.000 hipótesis estén balanceadas y efectivamente lo están asique solamente basto con agarrar las 300.000 primeras para entrenar los modelos con las distintas hipótesis procesadas.

In [ ]:
Counter(labels_train[0:300000])

Counter({'contradiction': 100057, 'entailment': 100165, 'neutral': 99778})

In [ ]:
scores = []
for index, p in parameters_preprocessing.iterrows():
    print(p.to_dict())
    score = crossval_score_parameters_preprocessing(text_train_valid[0:300000],labels_train_valid[0:300000],is_lc_=p['is_lc'],is_lem_=p['is_lem'],is_stop_=p['is_stop'],is_stem_=p['is_stem'],is_alpha_=p['is_alpha'])
    print(score)
    scores.append(score) 
    print('done.')
parameters_preprocessing['score'] = scores

In [ ]:
scores = []
for index, p in parameters_preprocessing.iterrows():
    print(p.to_dict())
    score = crossval_score_parameters_preprocessing_loaded(text_train_valid[0:300000],labels_train_valid[0:300000],is_lc_=p['is_lc'],is_lem_=p['is_lem'],is_stop_=p['is_stop'],is_stem_=p['is_stem'],is_alpha_=p['is_alpha'])
    print(score)
    scores.append(score) 
    print('done.')
parameters_preprocessing['score'] = scores

In [ ]:
parameters_preprocessing.sort_values('score',ascending=False)

,is_lc,is_lem,is_stop,is_stem,is_alpha,score
29,False,False,False,True,False,0.627573
13,True,False,False,True,False,0.627573
5,True,True,False,True,False,0.627403
21,False,True,False,True,False,0.627377
12,True,False,False,True,True,0.627240
28,False,False,False,True,True,0.627240
20,False,True,False,True,True,0.626947
4,True,True,False,True,True,0.626940
7,True,True,False,False,False,0.626157
23,False,True,False,False,False,0.625910


###Tomamos las 10 combinaciones que nos dieron mejores resultados y las evaluamos con todo el conjunto:

In [ ]:
best_parameters_preprocessing = parameters_preprocessing.sort_values('score',ascending=False)[0:10]
best_parameters_preprocessing

,is_lc,is_lem,is_stop,is_stem,is_alpha,score
29,False,False,False,True,False,0.627573
13,True,False,False,True,False,0.627573
5,True,True,False,True,False,0.627403
21,False,True,False,True,False,0.627377
12,True,False,False,True,True,0.627240
28,False,False,False,True,True,0.627240
20,False,True,False,True,True,0.626947
4,True,True,False,True,True,0.626940
7,True,True,False,False,False,0.626157
23,False,True,False,False,False,0.625910


In [ ]:
best_parameters_preprocessing = best_parameters_preprocessing.drop('score',axis=1)
best_parameters_preprocessing

,is_lc,is_lem,is_stop,is_stem,is_alpha
29,False,False,False,True,False
13,True,False,False,True,False
5,True,True,False,True,False
21,False,True,False,True,False
12,True,False,False,True,True
28,False,False,False,True,True
20,False,True,False,True,True
4,True,True,False,True,True
7,True,True,False,False,False
23,False,True,False,False,False


In [ ]:
#tardo  min s
scores = []
for index, p in best_parameters_preprocessing.iterrows():
    print(p.to_dict())
    score = crossval_score_parameters_preprocessing(text_train_valid,labels_train_valid,is_lc_=p['is_lc'],is_lem_=p['is_lem'],is_stop_=p['is_stop'],is_stem_=p['is_stem'],is_alpha_=p['is_alpha'])
    print(score)
    scores.append(score) 
    print('done.')
best_parameters_preprocessing['score'] = scores

In [ ]:
best_parameters_preprocessing.sort_values('score',ascending=False)

,is_lc,is_lem,is_stop,is_stem,is_alpha,score
29,False,False,False,True,False,0.631376
13,True,False,False,True,False,0.631376
5,True,True,False,True,False,0.631152
21,False,True,False,True,False,0.631140
7,True,True,False,False,False,0.630988
12,True,False,False,True,True,0.630982
28,False,False,False,True,True,0.630982
23,False,True,False,False,False,0.630864
4,True,True,False,True,True,0.630811
20,False,True,False,True,True,0.630793


##Conclusión con respecto a la elección de parámetros de procesamiento

Vemos que los 10 que dieron mejores combinaciones, ninguno de ellos les saca las stop words. Esto probablemente sea porque las stop words son las que le dan sentido a una oración, en este caso, a la hipótesis, por lo que son importantes para su clasificación.

Por otro lado, vemos poner todas las palabras en minúsculas puede ser verdadero o falso, vemos que no varía tanto la clasificación de hipótesis según se tenga las palabras en minúscula, mayúscula o mayúscula la primer letra y el resto en minúscula. Esto probablemente sea porque con mantener las stop words alcance para mantener el sentido de las hipótesis. 

En cuanto a la lematización, a pesar de que en principio creíamos que podía servir, vemos que varía, aunque las dos que dieron mejor resultado no hacen lematización. Sin embargo, vemos que las primeras 4 hacen stemming. Esto probablemente con que llevar a las palabras a su raíz, ayuda en la clasificación y que aunque a veces las lleve a raíces que en inglés no tendrían sentido, las oraciones pueden seguir manteniendo el sentido gracias a las stop words. También se ve, como los primeros cinco mejores resultados mantienen todo lo que no son palabras ya que al igual que las stop words ayudan a mantener el sentido de la hipótesis.

# NBMN

##Hiperparámetros Naive Bayes

###Creación de hiperparámetros

In [ ]:
random_seed = 0
hyperparameters_specs = { 
    'alpha':   [1e-10,1e-5,0.01,1,2],#0.01, 0.1, 1.0,10.0 #smooothing laplaciano (para sumar cuando frec=0), más grande, más parecido al uniforme
    'min_df':  [0.1,0.0001,0.00001,0.000002], #0.01, 0.1, 0.05, 0.49, 1, 10 #porcentaje mínimo de aparición en documentos
    'max_df':  [0.4, 0.8,0.9], # 0.5, 0.7,#porcentaje máximo de aparición en documentos
    'tf_idf':  [True, False], #usar tfidf o countVectorizer
}

In [ ]:
# DEFINING HYPERPARAMETERS: hacemos las combinaciones
def get_hyperparameters():
    #for isalpha in hyperparameters_specs['isalpha']:
      for tf_idf in hyperparameters_specs['tf_idf']:
          for min_df in hyperparameters_specs['min_df']:
              for max_df in hyperparameters_specs['max_df']:
                  for alpha in hyperparameters_specs['alpha']:
                      hyperparameters = {
                          #'isalpha': isalpha,
                          'alpha':   alpha,
                          'min_df':  min_df,
                          'max_df':  max_df,
                          'tf_idf':  tf_idf,
                      }
                      
                      yield hyperparameters #gives a generator function, para evitar muchos llamados al código base.

###Obtenemos hipotesis filtradas

Se evaluan los hiperparámetros para  los conjuntos a los que se les hace:
- Stemming
- Lowcase y stemming	

In [24]:
text_train_valid_filtrado_stem = open_txt('train_valid_filtrado_stem','/content/drive/Shareddrives/Redes neuronales/TP1/data_results/')

In [25]:
text_train_valid_filtrado_lc_stem = open_txt('train_valid_filtrado_lc_stem','/content/drive/Shareddrives/Redes neuronales/TP1/data_results/')

### Evalúo los hiperparametros




Evaluamos los hiperparámetros con dos tipos de procesamiento de textos, según los que nos dieron mejor resultado.

#### Para las hipotesis con stemming

In [ ]:
hyperparameters_stem = pd.DataFrame()

for hp in get_hyperparameters():
    df = pd.DataFrame(hp, index=[0])
    hyperparameters_stem = hyperparameters_stem.append(df,ignore_index=True) #ponemos combinaciones

hyperparameters_stem 

,alpha,min_df,max_df,tf_idf
0,1.000000e-10,0.100000,0.4,True
1,1.000000e-05,0.100000,0.4,True
2,1.000000e-02,0.100000,0.4,True
3,1.000000e+00,0.100000,0.4,True
4,2.000000e+00,0.100000,0.4,True
...,...,...,...,...
115,1.000000e-10,0.000002,0.9,False
116,1.000000e-05,0.000002,0.9,False
117,1.000000e-02,0.000002,0.9,False
118,1.000000e+00,0.000002,0.9,False


In [ ]:
scores = []
for index, hp in hyperparameters_stem.iterrows():
    print(hp.to_dict())
    score = run_sklearn_cross_val(text_train_valid_filtrado_stem,labels_train_valid,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters_stem['score'] = scores 

In [ ]:
hyperparameters_stem.sort_values('score',ascending=False)

,alpha,min_df,max_df,tf_idf,score
58,1.000000e+00,0.000002,0.9,True,0.639122
53,1.000000e+00,0.000002,0.8,True,0.639122
48,1.000000e+00,0.000002,0.4,True,0.639105
49,2.000000e+00,0.000002,0.4,True,0.636538
54,2.000000e+00,0.000002,0.8,True,0.636537
...,...,...,...,...,...
64,2.000000e+00,0.100000,0.4,False,0.352096
63,1.000000e+00,0.100000,0.4,False,0.352096
62,1.000000e-02,0.100000,0.4,False,0.352096
61,1.000000e-05,0.100000,0.4,False,0.352096


#### Para las hipotesis con lowcase y stemming

In [ ]:
hyperparameters = pd.DataFrame()

for hp in get_hyperparameters():
    df = pd.DataFrame(hp, index=[0])
    hyperparameters = hyperparameters.append(df,ignore_index=True) #ponemos combinaciones

hyperparameters 

,alpha,min_df,max_df,tf_idf
0,1.000000e-10,0.100000,0.4,True
1,1.000000e-05,0.100000,0.4,True
2,1.000000e-02,0.100000,0.4,True
3,1.000000e+00,0.100000,0.4,True
4,2.000000e+00,0.100000,0.4,True
...,...,...,...,...
115,1.000000e-10,0.000002,0.9,False
116,1.000000e-05,0.000002,0.9,False
117,1.000000e-02,0.000002,0.9,False
118,1.000000e+00,0.000002,0.9,False


In [ ]:
scores = []
for index, hp in hyperparameters.iterrows():
    print(hp.to_dict())
    score = run_sklearn_cross_val(text_train_valid_filtrado_lc_stem,labels_train_valid,hp)
    print(score)
    scores.append(score) 
    print('done.')
hyperparameters['score'] = scores 

In [ ]:
hyperparameters.sort_values('score',ascending=False)

,alpha,min_df,max_df,tf_idf,score
58,1.000000e+00,0.000002,0.9,True,0.639122
53,1.000000e+00,0.000002,0.8,True,0.639122
48,1.000000e+00,0.000002,0.4,True,0.639105
49,2.000000e+00,0.000002,0.4,True,0.636538
54,2.000000e+00,0.000002,0.8,True,0.636537
...,...,...,...,...,...
64,2.000000e+00,0.100000,0.4,False,0.352096
63,1.000000e+00,0.100000,0.4,False,0.352096
62,1.000000e-02,0.100000,0.4,False,0.352096
61,1.000000e-05,0.100000,0.4,False,0.352096


#Conclusión: modelo elegido

Como ambos dieron el mismo accuracy, se decidio el de stemming porque lleva menor procesamiento de textos. Se eligieron con los hiperparámetros que mejor accuracy dieron:
- alpha:1
- min_df: 0.000002
- max_df: 0.8
- con tfidf



##Métrica primaria: accuracy

El accuracy del modelo representa la proporción del número total de predicciones que fueron correctas:

$accuracy= \frac{TP+TN+FP+FN}{TP+TN}$

Siendo:
- TP: true positives
- TN: true negatives
- FP: false positives
- FN: false negatives

Es una de las métricas más utilizadas porque mide el porcentaje de casos que el modelo ha acertado. Se eligió como métrica primaria porque el dataset utilizado está balanceado por lo que da una buena idea de la calidad del modelo.
​


Evaluando con los parámetros indicados arriba, nos dió un accuracy de 	0,639122.

##Métricas secundarias

La precisión  mide el número de verdaderos positivos dividido por el número de falsos positivos más los verdaderos positivos. 

La exhaustividad o recall mide el número de verdaderos positivos dividido por el número de verdaderos positivos más falsos negativos. 

Es así como vemos que el recall nos da información sobre el rendimiento de un clasificador con respecto a falsos negativos (cuántos fallaron), mientras que la precisión nos proporciona información sobre su rendimiento con respecto a los falsos positivos (cuántos capturados).

Con Precisión se busca ser preciso (que estén bien clasificados), mientras que con Recall no se trata tanto de capturar casos correctamente sino más de capturar todos los casos en que se cumpla una clasificación. Entonces, básicamente, si queremos enfocarnos más en minimizar los falsos negativos, deseamos que nuestro recall sea lo más cercano posible al 100% sin que la precisión sea muy mala y si queremos enfocarnos en minimizar los falsos positivos, entonces nuestro enfoque debe orientarse a hacer que la Precisión sea lo más cercana posible al 100%.

El valor F1 se utiliza para combinar las medidas de precision y recall en un sólo valor. Esto es práctico porque hace más fácil el poder comparar el rendimiento combinado de la precisión  y la exhaustividad o recall entre varias soluciones. Esta es una de las ventajas del F1 score.

F1 se calcula haciendo la media armónica entre la precisión y la exhaustividad:

$F1 = 2 \cdot \frac{precision \cdot recall}{precision + recall}$

La desventaja de F1 es que asume que nos importa de igual forma la precisión y la exhaustividad. Esto no tiene que ser así en todos los problemas. Por ejemplo, cuando necesitamos predecir si hay riesgo de que un trozo de basura espacial se choque con un satélite, podemos valorar más la exhaustividad a riesgo de tener una peor precisión. 

El puntaje F se usa comúnmente para evaluar sistemas de recuperación de información, como motores de búsqueda, y también para muchos tipos de modelos de aprendizaje automático, en particular en el procesamiento del lenguaje natural, por esta razón elegimos incluirlo en nuestro análisis.

Por último, una curva ROC es una representación gráfica de la sensibilidad frente a la especificidad para un sistema clasificador binario. Ilustra en este tipo de sistemas el umbral de discriminación creado al graficar la tasa de verdaderos positivos frente a la tasa de falsos positivos. El roc_auc_score siempre va de 0 a 1 y clasifica las posibilidades predictivas. 0.5 es la línea de base para adivinar al azar, por lo que siempre debe estar por encima de 0.5. 

Para calcular roc_auc se eligio un paramatero multi_class='ovr', ya que se considerá que este estaría dando el resultado más general posible. Ya que la clasificación binaria esta dada por la clase que se quiere evaluar contra una "clase" donde están el resto de las las clases.

En cuanto a los parámetros de precision score, usamos 'macro', que calcula métricas para cada etiqueta y encuentre su media no ponderada (no tiene en cuenta el desequilibrio de etiquetas) en vez de 'micro', (que calcula métricas globalmente considerando cada elemento de la matriz de indicadores de etiquetas como una etiqueta).



In [22]:
text_train_valid_filtrado_stem = open_txt('train_valid_filtrado_stem','/content/drive/Shareddrives/Redes neuronales/TP1/data_results/')

In [23]:
metric_results = obtain_metrics(text_train_valid_filtrado_stem,labels_train_valid,tf_idf_=True,alpha_ = 1,min_df_= 0.000002, max_df_=0.8, ngram_range_ = (1,2))
metric_results

,Accuracy,Precision,Recall,F1 score,ROC_AUC
0,0.639122,0.639829,0.639086,0.638906,0.815781


##Predición de labels

In [24]:
text_test_filtrado = procesamiento_textos(text_test,text_name='test',is_lc=False,is_lem=False,is_stop=False,is_stem=True,is_alpha=False,path='/content/drive/Shareddrives/Redes neuronales/TP1/data_results/')
text_test_filtrado[0:5]

['the church has crack in the ceil .',
 'the church is fill with song .',
 'a choir sing at a basebal game .',
 'the woman is young .',
 'the woman is veri happi .']

In [25]:
labels_test = predict_label(text_train_valid_filtrado_stem,labels_train_valid,text_test_filtrado,tf_idf_=True,alpha_ = 1,min_df_=0.000002, max_df_=0.8)
labels_test

,1,2,3,4,5,pred_labels
pairID,,,,,,
0,contradiction,contradiction,contradiction,neutral,neutral,contradiction
1,contradiction,neutral,neutral,contradiction,neutral,neutral
2,neutral,neutral,neutral,neutral,neutral,neutral
3,entailment,entailment,entailment,entailment,entailment,entailment
4,neutral,neutral,neutral,neutral,neutral,neutral
...,...,...,...,...,...,...
9819,entailment,entailment,entailment,contradiction,entailment,entailment
9820,entailment,entailment,entailment,entailment,entailment,entailment
9821,contradiction,contradiction,contradiction,contradiction,contradiction,contradiction


#Técnica de validación: cross validation

El método de validación Hold-out divide su conjunto de datos en un conjunto de "entrenamiento" y "prueba". EN el primero se entrena el modelo, y el segundo se usa para ver qué tan bien se desempeña ese modelo en datos no vistos. 
Validación cruzada
La validación cruzada o "validación cruzada de k veces" consta de la subdivisión aleatoria del conjunto de datos en "k" grupos. Uno de los grupos se utiliza como conjunto de prueba y el resto se utiliza como conjunto de entrenamiento. El modelo se entrena en el conjunto de entrenamiento y se puntúa en el conjunto de prueba. Luego, el proceso se repite hasta que cada grupo único se haya utilizado como conjunto de prueba. 
Por ejemplo, para una validación cruzada de 5 veces, el conjunto de datos se dividiría en 5 grupos, y  Esto se puede ver en el gráfico siguiente.

La validación cruzada suele ser el método preferido porque le da al modelo la oportunidad de entrenar en múltiples divisiones de prueba del conjunto de entrenamiento. Esto brinda una mejor indicación de qué tan bien funcionará el modelo con datos no vistos. Hold-out, por otro lado, depende de una sola división de prueba de entrenamiento. Eso hace que la puntuación de este método dependa de cómo se dividen los datos en conjuntos de prueba y de entrenamiento. 

Es por esto, que para Naive Bayes Multinomial se decidió utilizar cross validation ya que da resultados más confiables. En este caso, se decidión hacer cross validation con k=5 para obtener los scores.

#Conclusión obtenida de ambos modelos

Al comparar las metricas obtenidas entre ambos modelos, se que el MLP discrimina significativamente mejor, esto lo muestra el  roc_auc, que muestra claramente la capacidad discriminativa del modelo. Sin embrago, entrenar el modelo de MLP mucho tarda más que el Multinomial Bayes.

Otra cosa a tener en cuenta, es que el MLP requiere mucha más memoria que con Bayes debido a su complejidad. Dado, que la diferencia de scores entre metricas no es lo significativamente mayor, no se justifica utilizar un MLP para la clasificación de este problema.

Por otro lado, en cuanto al sesgo en los datos, en una primera instancia, vimos que el conjunto de datos presentaba un valor de accuracy del 60% sin hacer preprocesamiento alguno. Esto demuestra un sesgo debido a que, como estamos trabajando con tres clases diferentes (contradiction, entailment y neutral) lo que debería ocurrir es un accuracy inicial cercano el 33% (igual de probable para las tres clases).
Este sesgo puede deberse a que, como se escribe en el paper provisto por la cátedra, el conjunto de datos presenta ciertas particularidades que inicialmente facilitan la clasificación. Entre ellas son la importante presencia de ciertas palabras y la longitud de la oración. Procedemos a explicar en mayor detalle para cada una de las clases.
En cuanto a la clase entailment, esta cuenta con una amplia gama de palabras genéricas, aproximaciones numéricas y eliminación de géneros, así como también cierta vaguedad en la descripción de lugares; en cuanto a nuetral, hay una presencia importante de adjetivos modifiadores y superlativos, es decir, palabras que introducen información que no está obviamente implícita en la premisa, pero que es plausible.Por último, en cuanto a contradiction, hay una fuerte presencia de palabras de negación (como nadie, no, nunca y nada) así como palabras que no son inherentemente negativas pero sin contradictorias a muchos escenarios (como dormir, que contradice cualquier otra actividad y desnudo, quecontradice cualquier descripción de la ropa).

Tras ejecutar el algoritmo de cross-validation se obtuvo, con Naïve Bayes, un accuracy de 0.639122 lo cual no representó un cambio significativo respecto a la clasificación de entrada.

#resultados de NBMN con solamente un modelo

In [26]:
text_train_filtered = text_train_valid_filtrado_stem[0:549367] #obtenemos los datos de train
text_valid_filtered = text_train_valid_filtrado_stem[549367:] #obtenemos los datos de valid

Entrenamos un modelo con las hipotesis de train filtradas y obtenemos los resultados de submit

In [ ]:
results = run_NBMN_test(text_train_filtered,labels_train,text_valid_filtered,labels_val,text_test_filtrado,tf_idf_=True,alpha_ = 1,min_df_=0.000002, max_df_=0.8, ngram_range_ = (1,2))
results

training model...
evaluating hyperparameters...


,pred_labels
pairID,
0,contradiction
1,neutral
2,neutral
3,entailment
4,neutral
...,...
9819,entailment
9820,entailment
9821,contradiction
